# Installations

In [1]:
 !pip install neo4j openai langchain
 !pip install getpass4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
  Created wheel for neo4j: filename=neo4j-5.14.0-py3-none-any.whl size=265476 sha256=6b5b718af01bf2c61a0678afbc9b30d1033ca9e432bd41d2f38ab3005aae234e
  Stored in directory: /root/.cache/pip/wheels/53/e0/d7/603097e3fed62f821523433801c09e04cd7a7610c7565bd5a3
Successfully built neo4j
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of 

# Importing packages

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
import getpass

# Connecting to Neo4j Database

In [3]:
graph = Neo4jGraph(
    url="bolt://44.211.132.3:7687",
    username="neo4j",
    password="wing-rehabilitation-screw"
)

# Set up Environment Variables

In [4]:
import os
name = 'OPENAI_API_KEY'
secret = getpass.getpass("Enter the OpenAI API Key")
os.environ[name] = secret
print(f"Have successfully set up {name} as an environment variable.")

Enter the OpenAI API Key··········
Have successfully set up OPENAI_API_KEY as an environment variable.


# Querying the graph database to return answers to natural text

In [5]:
graph.refresh_schema()

In [6]:
print(graph.schema)


        Node properties are the following:
        [{'labels': 'Loc', 'properties': [{'property': 'name', 'type': 'STRING'}]}]
        Relationship properties are the following:
        [{'properties': [{'property': 'cost', 'type': 'INTEGER'}], 'type': 'ROAD'}]
        The relationships are the following:
        ['(:Loc)-[:ROAD]->(:Loc)']
        


In [7]:
# Setting up ChatGPT integration with Graph Database
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,
)

# Basic operations

### 1. Find nodes and edges.

In [ ]:
chain.run("""
How many nodes does the graph have?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n) RETURN count(n)
Full Context:
[{'count(n)': 6}]

> Finished chain.


'The graph has 6 nodes.'

In [ ]:
chain.run("""
How many edges does the graph have?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH ()-[:ROAD]->() 
RETURN count(*) AS edge_count
Full Context:
[{'edge_count': 10}]

> Finished chain.


'The graph has 10 edges.'

In [ ]:
chain.run("""
How many nodes and edges does the graph have?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n) 
RETURN count(n) AS nodes, size((:Loc)-[:ROAD]->(:Loc)) AS edges


ValueError: ignored

**Finding**: The LLM can find nodes and edges if prompted seperately but not together.

### 2. Finding the degree of each node.

In [ ]:
chain.run("""
What is the degree of node a in the graph? Allow it to be case-insensitive.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Loc)-[:ROAD]->()
WHERE toLower(a.name) = 'a'
RETURN count(*) AS degree
Full Context:
[{'degree': 3}]

> Finished chain.


'The degree of node a in the graph is 3.'

In [ ]:
chain.run("""
What is the degree of all nodes in the graph? Use count function.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Loc)-[r:ROAD]->(m:Loc)
RETURN n.name, count(r), m.name
Full Context:
[{'n.name': 'A', 'count(r)': 1, 'm.name': 'D'}, {'n.name': 'A', 'count(r)': 1, 'm.name': 'C'}, {'n.name': 'A', 'count(r)': 1, 'm.name': 'B'}, {'n.name': 'B', 'count(r)': 1, 'm.name': 'D'}, {'n.name': 'C', 'count(r)': 1, 'm.name': 'E'}, {'n.name': 'C', 'count(r)': 1, 'm.name': 'D'}, {'n.name': 'D', 'count(r)': 1, 'm.name': 'D'}, {'n.name': 'D', 'count(r)': 1, 'm.name': 'F'}, {'n.name': 'D', 'count(r)': 1, 'm.name': 'E'}, {'n.name': 'E', 'count(r)': 1, 'm.name': 'F'}]


KeyboardInterrupt: ignored

In [ ]:
chain.run("""
What is the degree of all nodes and its edges in the graph? Use count function. To do this, first match all starting nodes. Then, use count function to find all edges linked to that node.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Loc)
RETURN n.name, count((n)-[:ROAD]->()) AS degree


ValueError: ignored

**Findings**: LLM can find the degree if the individual node is specified. Few-shot learning was attempted to see if LLM can learn how to generate correct prompt.

**Correct prompt**:

```
MATCH (a:Loc)-[r:ROAD]->()

RETURN a.name, count(r) AS degree
```



### 3. Find indegree and outdegree centrality measures

#### For individual nodes.

In [ ]:
chain.run("""
Find the in degree centrality of node A. Use count function.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Loc)-[:ROAD]->(:Loc)
WHERE a.name = 'A'
RETURN count(*) AS inDegreeCentrality
Full Context:
[{'inDegreeCentrality': 3}]

> Finished chain.


'The in degree centrality of node A is 3.'

In [ ]:
chain.run("""
Find the number of incoming links to node A. Use count function.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Loc)-[:ROAD]->(b:Loc)
WHERE b.name = 'A'
RETURN count(a)
Full Context:
[{'count(a)': 0}]

> Finished chain.


'The number of incoming links to node A is 0.'

In [ ]:
chain.run("""
Find the indegree centrality of node A. Use count function. Indegree centrality is the number of incoming edges to a node.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Loc)-[:ROAD]->(:Loc)
WHERE a.name = 'A'
RETURN count(*) AS indegree_centrality
Full Context:
[{'indegree_centrality': 3}]

> Finished chain.


'The indegree centrality of node A is 3. This means that there are 3 incoming edges to node A.'

In [ ]:
chain.run("""
Find the outdegree centrality of node A. Use count function.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Loc)-[:ROAD]->(b:Loc)
WHERE a.name = 'A'
RETURN count(b) AS outdegree_centrality
Full Context:
[{'outdegree_centrality': 3}]

> Finished chain.


'The outdegree centrality of node A is 3.'

**Findings**:

1. It cannot find the indegree centrality but able to find the number of incoming links.
2. It is able to find the outdegree centrality.

#### For all nodes.

In [ ]:
chain.run("""
Find the indegree centrality of all nodes. Use count function.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Loc)
RETURN n.name, count(()-[:ROAD]->(n)) AS indegree_centrality


ValueError: ignored

In [ ]:
chain.run("""
Find the number of incoming links to all nodes. Use count function.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Loc)<-[:ROAD]-()
RETURN n.name, count(*) AS incoming_links
Full Context:
[{'n.name': 'B', 'incoming_links': 1}, {'n.name': 'C', 'incoming_links': 1}, {'n.name': 'D', 'incoming_links': 4}, {'n.name': 'E', 'incoming_links': 2}, {'n.name': 'F', 'incoming_links': 2}]

> Finished chain.


'The count function can be used to find the number of incoming links to all nodes.'

In [ ]:
chain.run("""
Indegree centrality is the number of incoming edges to a node. Find the indegree centrality of all nodes. Use count function.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Loc)
RETURN n.name, count(()-[:ROAD]->(n)) AS indegree_centrality


ValueError: ignored

In [ ]:
chain.run("""
Find the outdegree centrality of all nodes. Use count function.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Loc)-[r:ROAD]->()
RETURN n.name, count(r) AS outdegree_centrality
Full Context:
[{'n.name': 'A', 'outdegree_centrality': 3}, {'n.name': 'B', 'outdegree_centrality': 1}, {'n.name': 'C', 'outdegree_centrality': 2}, {'n.name': 'D', 'outdegree_centrality': 3}, {'n.name': 'E', 'outdegree_centrality': 1}]

> Finished chain.


'The outdegree centrality of all nodes can be found using the count function.'

**Findings**:

1. It cannot find the indegree centrality but able to find the number of incoming links for all.
2. It is able to find the outdegree centrality for all nodes.

Correct Cypher queries:


```
`# Indegree centrality`
MATCH ()-[r:ROAD]->(a:Loc)
RETURN a.name, count(r) AS indegree

`# Outdegree centrality`
MATCH (a:Loc)-[r:ROAD]->()
RETURN a.name, count(r) AS outdegree

```





### 4. Find betweenness centrality measures

#### For individual nodes.

In [ ]:
chain.run("""
Find the in betweenness centrality of node A.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Loc {name: 'A'})
CALL algo.betweenness.stream('Loc', 'ROAD', {direction: 'both'})
YIELD nodeId, centrality
WHERE id(a) = nodeId
RETURN centrality


ClientError: ignored

**Findings**:

1. It is able to identify the correct query (direction: 'out' is missed).

#### For all nodes.

In [ ]:
chain.run("""
Find the in betweenness centrality of all nodes.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
CALL algo.betweenness.stream('Loc', 'ROAD', {direction: 'both'})
YIELD nodeId, centrality
RETURN algo.getNodeById(nodeId).name AS Node, centrality AS BetweennessCentrality


ClientError: ignored

**Findings**:

1. It can find the correct query.

Correct Cypher queries:


```
`# Betweenness centrality`
CALL algo.betweenness.stream('Loc', 'ROAD', {direction: 'out'})
YIELD nodeId, centrality
RETURN algo.getNodeById(nodeId).name AS Node, centrality AS BetweennessCentrality

```





### 5. Are there any isolated nodes?

In [ ]:
chain.run("""
Are there any isolated nodes in the network? Can you name all?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Loc)
WHERE NOT (n)-[:ROAD]-()
RETURN n.name
Full Context:
[{'n.name': 'G'}]

> Finished chain.


'Yes, there are isolated nodes in the network. The only isolated node in the network is node G.'

**Findings**:

1. It can successfully identify the isolated nodes.

### 6. Is there a complete graph?

In [ ]:
chain.run("""
Find all graphs in which each pair of nodes is connected by an edge.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n1:Loc)-[:ROAD]->(n2:Loc)
RETURN n1, n2
Full Context:
[{'n1': {'name': 'A'}, 'n2': {'name': 'B'}}, {'n1': {'name': 'A'}, 'n2': {'name': 'C'}}, {'n1': {'name': 'D'}, 'n2': {'name': 'D'}}, {'n1': {'name': 'C'}, 'n2': {'name': 'D'}}, {'n1': {'name': 'B'}, 'n2': {'name': 'D'}}, {'n1': {'name': 'A'}, 'n2': {'name': 'D'}}, {'n1': {'name': 'D'}, 'n2': {'name': 'E'}}, {'n1': {'name': 'C'}, 'n2': {'name': 'E'}}, {'n1': {'name': 'E'}, 'n2': {'name': 'F'}}, {'n1': {'name': 'D'}, 'n2': {'name': 'F'}}]

> Finished chain.


'Based on the provided information, there are several graphs in which each pair of nodes is connected by an edge. Here are the graphs that satisfy this condition:\n\n1. Graph with nodes A, B, and C: \n   - A is connected to B\n   - A is connected to C\n\n2. Graph with nodes D and E:\n   - D is connected to D (self-loop)\n   - D is connected to E\n\n3. Graph with nodes B, C, and D:\n   - B is connected to D\n   - C is connected to D\n\n4. Graph with nodes A, D, and F:\n   - A is connected to D\n   - D is connected to F\n\n5. Graph with nodes C, D, and E:\n   - C is connected to D\n   - C is connected to E\n\n6. Graph with nodes E and F:\n   - E is connected to F\n\nThese are all the graphs in which each pair of nodes is connected by an edge based on the given information.'

**Findings**:

1. Need to be informed what is a complate graph.

### 7. Is there a clique?

In [8]:
chain.run("""
Find all cliques in the graph.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n1:Loc)-[r1:ROAD]->(n2:Loc)-[r2:ROAD]->(n3:Loc)-[r3:ROAD]->(n1)
RETURN n1, n2, n3
Full Context:
[]

> Finished chain.


"I'm sorry, but without any information about the graph, I am unable to find all the cliques. Could you please provide more details or specifications about the graph?"

In [10]:
chain.run("""
Find the largest subset of nodes that are directly and completely connected to the rest of the set.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Loc)-[:ROAD]->(m:Loc)
WITH n, count(m) as connections
ORDER BY connections DESC
LIMIT 1
MATCH (n)-[:ROAD]->(m:Loc)
RETURN collect(m) as largest_subset
Full Context:
[{'largest_subset': [{'name': 'D'}, {'name': 'C'}, {'name': 'B'}]}]

> Finished chain.


'The largest subset of nodes that are directly and completely connected to the rest of the set is composed of nodes D, C, and B.'

In [14]:
chain.run("""
A clique is the largest subset of nodes that are directly and completely connected to the rest of the set. Find all the cliques in the graph? Use count function.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Loc)-[:ROAD]->(m:Loc)
WITH n, count(m) as numConnections
MATCH (n)-[:ROAD]->(m:Loc)
WITH n, collect(m) as connections, numConnections
WHERE size(connections) = numConnections
RETURN connections
Full Context:
[{'connections': [{'name': 'D'}, {'name': 'C'}, {'name': 'B'}]}, {'connections': [{'name': 'D'}, {'name': 'F'}, {'name': 'E'}]}, {'connections': [{'name': 'E'}, {'name': 'D'}]}, {'connections': [{'name': 'D'}]}, {'connections': [{'name': 'F'}]}]

> Finished chain.


"I'm sorry, but I don't have enough information to answer your question."

**Findings**:
1. Cliques need to be defined to the LLM.
2. Preference towards using size() function which does not exist. It needs to be mentioned to use count().

### 8. What is the density of this graph?

findings:
1. Without telling LLM the formula of density, the query and answer is wrong.
2. After including the formula of density, LLM is able to generate the correct query. However, it will round the density to an integer which is not we want. (Density is always inside [0,1]
3. After including the formula of density and tell LLM to return a decimal, the answer is correct.

In [ ]:
chain.run("""
Calculate the density of this graph.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (l1:Loc)-[r:ROAD]->(l2:Loc)
RETURN count(r) / (count(DISTINCT l1) * count(DISTINCT l2)) AS density
Full Context:
[{'density': 0}]

> Finished chain.


'The density of the graph is 0.'

In [ ]:
chain.run("""
Calculate the density of this graph. (the formula of density is: density = observed number of edges/maximum possible edges).
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH ()-[:ROAD]->() 
WITH count(*) AS observed_edges
MATCH (l:Loc)
WITH count(l) AS num_nodes, observed_edges
RETURN observed_edges / (num_nodes * (num_nodes - 1)) AS density
Full Context:
[{'density': 0}]

> Finished chain.


"I'm sorry, but I don't have enough information to calculate the density of the graph."

In [ ]:
chain.run("""
Calculate the density of this graph. (the formula of density is: density = observed number of edges/maximum possible edges).return the number in decimal
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH ()-[:ROAD]->() 
WITH count(*) AS observedEdges
MATCH (l:Loc)
WITH observedEdges, count(l) AS totalLocs
RETURN toFloat(observedEdges) / (toFloat(totalLocs) * (toFloat(totalLocs) - 1)) AS density
Full Context:
[{'density': 0.23809523809523808}]

> Finished chain.


'The density of the graph is 0.23809523809523808.'

### 9. Reciprocity

Finding: Similar to density, we need to explicitly specify  "return the answer in decimal" to get the desired to get the desire result.

In [ ]:
chain.run("""
Calculate the reciprocity of this database.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Loc)-[r:ROAD]->(b:Loc)
WITH COUNT(*) AS total, SUM(CASE WHEN (b)-[:ROAD]->(a) THEN 1 ELSE 0 END) AS reciprocal
RETURN reciprocal / total AS reciprocity
Full Context:
[{'reciprocity': 0}]

> Finished chain.


'The reciprocity of this database is 0.'

In [ ]:
chain.run("""
Calculate the reciprocity of this database. return me in decimal
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Loc)-[r:ROAD]->(b:Loc)
WITH COUNT(*) AS total, SUM(CASE WHEN (b)-[:ROAD]->(a) THEN 1 ELSE 0 END) AS reciprocal
RETURN reciprocal / toFloat(total) AS reciprocity
Full Context:
[{'reciprocity': 0.1}]

> Finished chain.


'The reciprocity of this database is 0.1.'

### 10. Global Clustering Coefficient

Findings
1. When there is isolated nodes in network, calculation will fail due to divide by zero error. (the degree of an isolated node is zero)
2. With specifying exclude isolated nodes, the query still contains error.

In [ ]:
chain.run("""
Calculate the Average Clustering Coefficient of this network.
""")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Loc)-[r:ROAD]->(m:Loc)
WITH n, count(r) as degree
MATCH (n)-[:ROAD]->(m)-[:ROAD]->(n)
RETURN sum(degree) / (count(n) * (count(n) - 1)) as clustering_coefficient


ClientError: ignored

In [ ]:
chain.run("""
Calculate the Average Clustering Coefficient of this network. Exclude isolated nodes.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Loc)-[r:ROAD]->(m:Loc)
WITH n, count(DISTINCT m) as degree
WHERE degree > 1
MATCH (n)-[:ROAD]->(m)-[:ROAD]->(o)
WHERE n <> o
WITH n, count(DISTINCT o) as triangles
RETURN sum(triangles) / sum(degree * (degree - 1)) as clustering_coefficient


ValueError: ignored

**Findings**:

1. Need to be informed what is a complete graph.

### 11. Find the path length.

In [19]:
chain.run("""
Find the path length for Node A from Node C.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH path = (a:Loc {name: 'A'})-[:ROAD*]->(c:Loc {name: 'C'})
RETURN length(path) AS path_length
Full Context:
[{'path_length': 1}]

> Finished chain.


'The path length for Node A from Node C is 1.'

**Findings:**
1. It treats the graph as an unweighted graph.

### 12. Find the local clustering coefficient.

#### Single Node

In [23]:
chain.run("""
Find the local clustering coefficient of all nodes. Exclude isolated nodes. Local clustering coefficient is the triangle count.
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n:Loc)-[:ROAD]->(m:Loc)-[:ROAD]->(o:Loc)-[:ROAD]->(n)
WHERE id(n) < id(m) AND id(m) < id(o)
RETURN n.name AS Node, count(*) AS TriangleCount
ORDER BY TriangleCount DESC
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have the information to answer your question."

**Findings:**
1. It needs context on what is local clustering effect.